In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [3]:
# Expressoes regulares comuns
REGEX_NOT_NUM = r'^[^0-9]*$'

In [4]:
# Criar o contexto do spark
sc = SparkContext()

# Instancia o criador de sessao do spark
spark = (SparkSession.builder
                     .master("local[7]")
                     .appName("Aceleração PySpark - Capgemini")
                    ).getOrCreate()

In [5]:
def split_csv(line):
    return tuple(map(lambda x: x.replace('"',''), line.split(",")))

In [6]:
schema_airports = StructType([
    StructField("faa",  StringType(),  True),
    StructField("name", StringType(),  True),
    StructField("lat",  FloatType(),   True),
    StructField("lon",  FloatType(),   True),
    StructField("alt",  IntegerType(), True),
    StructField("tz",   FloatType(), True),
    StructField("dst",  StringType(),  True)
])

schema_planes = StructType([
    StructField("tailnum",      StringType(),  True),
    StructField("year",         IntegerType(), True),
    StructField("type",         StringType(),  True),
    StructField("manufacturer", StringType(),  True),
    StructField("model",        StringType(),  True),
    StructField("engines",      IntegerType(), True),
    StructField("seats",        IntegerType(), True),
    StructField("speed",        IntegerType(), True),
    StructField("engine",       StringType(),  True)
])

schema_flights = StructType([
    StructField("year",      IntegerType(), True),
    StructField("month",     IntegerType(), True),
    StructField("day",       IntegerType(), True),
    StructField("dep_time",  StringType(),  True),
    StructField("dep_delay", IntegerType(), True),
    StructField("arr_time",  StringType(),  True),
    StructField("arr_delay", IntegerType(), True),
    StructField("carrier",   StringType(),  True),
    StructField("tailnum",   StringType(),  True),
    StructField("flight",    StringType(),  True),
    StructField("origin",    StringType(),  True),
    StructField("dest",      StringType(),  True),
    StructField("air_time",  IntegerType(), True),
    StructField("distance",  IntegerType(), True),
    StructField("hour",      IntegerType(), True),
    StructField("minute",    IntegerType(), True),
])

In [7]:
#voltar
df_airports = (spark.read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_airports)
                  .load("data/airports.csv"))

df_planes = (spark.read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_planes)
                  .load("data/planes.csv"))

df_flights = (spark.read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_flights)
                  .load("data/flights.csv"))

In [8]:
rdd_airports = sc.textFile("/airports.csv")
rdd_planes   = sc.textFile("data/planes.csv")

In [9]:
# armazena a primeira linha do arquivo como referencia
header_airports = rdd_airports.first()
header_planes   = rdd_planes.first()

# Remove a primeira linha do arquivo
rdd_airports = rdd_airports.filter(lambda line: line != header_airports).map(split_csv)
rdd_planes   = rdd_planes.filter(lambda   line: line != header_planes).map(split_csv)


## Airports Dataset

### Pergunta 1

In [10]:
df_airports.filter(F.col('alt') < 0 ).show()
df_airports = df_airports.withColumn("alt", 
                                     F.when(F.col("alt") < 0 , 0)
                                     .otherwise(F.col("alt"))
                                    )

df_airports.filter(F.col('alt') < 0 ).show()

+---+-------------+---------+----------+---+----+---+
|faa|         name|      lat|       lon|alt|  tz|dst|
+---+-------------+---------+----------+---+----+---+
|IPL|  Imperial Co| 32.83422|-115.57874|-54|-8.0|  A|
|NJK|El Centro Naf|32.829224|-115.67167|-42|-8.0|  A|
+---+-------------+---------+----------+---+----+---+

+---+----+---+---+---+---+---+
|faa|name|lat|lon|alt| tz|dst|
+---+----+---+---+---+---+---+
+---+----+---+---+---+---+---+



 **Utilizando RDD**

In [11]:
%%time
def alt(row):
    if (int(row[4]) < 0):
        row = (row[0], row[1], row[2], row[3], 0, row[5], row[6])
    return row

rdd_airports = rdd_airports.map(alt)
rdd_airports.take(6)

Wall time: 776 ms


[('04G', 'Lansdowne Airport', '41.1304722', '-80.6195833', '1044', '-5', 'A'),
 ('06A',
  'Moton Field Municipal Airport',
  '32.4605722',
  '-85.6800278',
  '264',
  '-5',
  'A'),
 ('06C', 'Schaumburg Regional', '41.9893408', '-88.1012428', '801', '-6', 'A'),
 ('06N', 'Randall Airport', '41.431912', '-74.3915611', '523', '-5', 'A'),
 ('09J',
  'Jekyll Island Airport',
  '31.0744722',
  '-81.4277778',
  '11',
  '-4',
  'A'),
 ('0A9',
  'Elizabethton Municipal Airport',
  '36.3712222',
  '-82.1734167',
  '1593',
  '-4',
  'A')]

### Pergunta 2 

In [12]:
df_airports.filter(F.col('tz').between(-7,-5)).show()
print(df_airports.filter(F.col('tz').between(-7,-5)).count())
print(df_airports.filter(F.col('dst') == 'A').count())

df_airports = df_airports.withColumn("dst", 
                                    F.when(F.col("tz").between(-7,-5), 'A')
                                   .otherwise(F.col("dst")))

df_airports.filter((F.col('tz').between(-7,-5)) & (F.col('dst') != 'A')).show()
print(df_airports.filter(F.col('dst') == 'A').count())

+---+--------------------+---------+----------+----+----+---+
|faa|                name|      lat|       lon| alt|  tz|dst|
+---+--------------------+---------+----------+----+----+---+
|04G|   Lansdowne Airport|41.130474| -80.61958|1044|-5.0|  A|
|06A|Moton Field Munic...| 32.46057| -85.68003| 264|-5.0|  A|
|06C| Schaumburg Regional| 41.98934| -88.10124| 801|-6.0|  A|
|06N|     Randall Airport| 41.43191| -74.39156| 523|-5.0|  A|
|0G6|Williams County A...|41.467304|-84.506775| 730|-5.0|  A|
|0G7|Finger Lakes Regi...|42.883564|-76.781235| 492|-5.0|  A|
|0P2|Shoestring Aviati...|39.794823|-76.647194|1000|-5.0|  U|
|0W3|Harford County Ai...|39.566837|  -76.2024| 409|-5.0|  A|
|10C|  Galt Field Airport| 42.40289|-88.375114| 875|-6.0|  U|
|17G|Port Bucyrus-Craw...|40.781555| -82.97481|1003|-5.0|  A|
|1B9| Mansfield Municipal|42.000134| -71.19677| 122|-5.0|  A|
|1CS|Clow Internationa...|41.695976|-88.129234| 670|-6.0|  U|
|1OH|     Fortman Airport|40.555325| -84.38662| 885|-5.0|  U|
|1RL|Poi

**Utilizando RDD**

In [13]:
def tz_replace_dst(row):
    if (-5 >= int(row[5]) >= -7 ) :
        row = row[:6] + ('A',)
    return row

rdd_airports = rdd_airports.map(tz_replace_dst)
rdd_airports.take(6)

[('04G', 'Lansdowne Airport', '41.1304722', '-80.6195833', '1044', '-5', 'A'),
 ('06A',
  'Moton Field Municipal Airport',
  '32.4605722',
  '-85.6800278',
  '264',
  '-5',
  'A'),
 ('06C', 'Schaumburg Regional', '41.9893408', '-88.1012428', '801', '-6', 'A'),
 ('06N', 'Randall Airport', '41.431912', '-74.3915611', '523', '-5', 'A'),
 ('09J',
  'Jekyll Island Airport',
  '31.0744722',
  '-81.4277778',
  '11',
  '-4',
  'A'),
 ('0A9',
  'Elizabethton Municipal Airport',
  '36.3712222',
  '-82.1734167',
  '1593',
  '-4',
  'A')]

### Pergunta 3

In [14]:
print(df_airports.filter(F.col('dst') == 'U').count())
df_airports.filter(F.col('dst') == 'U').show()

df_airports = df_airports.withColumn("dst", 
                                     F.when(F.col("dst") == 'U', 'A')
                                     .otherwise(F.col("dst")))

df_airports.filter(F.col('dst') == 'U').show()
print(df_airports.filter(F.col('dst') == 'U').count())
print(df_airports.filter(F.col('dst') == 'A').count())

8
+---+--------------------+---------+----------+----+-----+---+
|faa|                name|      lat|       lon| alt|   tz|dst|
+---+--------------------+---------+----------+----+-----+---+
|19A|Jackson County Ai...|34.175865|  -83.5616| 951| -4.0|  U|
|BLD|Boulder City Muni...|  35.5651|  -114.514|2201| -8.0|  U|
|GCW|Grand Canyon West...|  35.5925| -113.4859|4825| -8.0|  U|
|MXY|    McCarthy Airport| 61.43706|-142.90308|1531| -8.0|  U|
|NGZ|         NAS Alameda|  37.7861| -122.3186|  10| -9.0|  U|
|SWD|      Seward Airport|60.126938|-149.41881|  22| -8.0|  U|
|SXQ|    Soldotna Airport|60.474957|-151.03824| 113| -8.0|  U|
|WWT|      Newtok Airport|60.939167|-164.64111|  25|-10.0|  U|
+---+--------------------+---------+----------+----+-----+---+

+---+----+---+---+---+---+---+
|faa|name|lat|lon|alt| tz|dst|
+---+----+---+---+---+---+---+
+---+----+---+---+---+---+---+

0
1388


In [15]:
#rdd

def dst(row):
    if(row[6] == 'U'):
        row = row[:6] + ('A',)
    return row

rdd_airports = rdd_airports.map(dst)
rdd_airports.take(6)

[('04G', 'Lansdowne Airport', '41.1304722', '-80.6195833', '1044', '-5', 'A'),
 ('06A',
  'Moton Field Municipal Airport',
  '32.4605722',
  '-85.6800278',
  '264',
  '-5',
  'A'),
 ('06C', 'Schaumburg Regional', '41.9893408', '-88.1012428', '801', '-6', 'A'),
 ('06N', 'Randall Airport', '41.431912', '-74.3915611', '523', '-5', 'A'),
 ('09J',
  'Jekyll Island Airport',
  '31.0744722',
  '-81.4277778',
  '11',
  '-4',
  'A'),
 ('0A9',
  'Elizabethton Municipal Airport',
  '36.3712222',
  '-82.1734167',
  '1593',
  '-4',
  'A')]

### Pergunta 4

In [16]:
df_airports = df_airports.withColumn("region", 
                                    F.when(F.col("lon") < -124, 'ALASKA')
                                     .when((F.col("lon") > -50) | (F.col("lat") < 24),'OFFSHORE')
                                     .when( F.col("lon").between(-124,-95), 'MAINLAND-WEST')
                                     .when( F.col("lon").between(-95, -50), 'MAINLAND-EAST')
                                     .otherwise('NaN'))

df_airports.groupBy('region').count().show()

+-------------+-----+
|       region|count|
+-------------+-----+
|       ALASKA|  261|
|     OFFSHORE|    4|
|MAINLAND-EAST|  696|
|MAINLAND-WEST|  436|
+-------------+-----+



### Pergunta 5

In [17]:
AP_LIST = [ "Airport", "Tradeport", "Heliport", "Airpor","Arpt"]
REGEX_AP = r'|'.join(map(lambda word: f'.*(^| ){word}( |$).*', AP_LIST))

df_airports = df_airports.withColumn("type",
                                    F.when(F.col("name").rlike(REGEX_AP), 'AP')
                                     .when(F.col("name").contains("Aerodrome"), 'AD')
                                     .when(F.col("name").contains("Airpark") | F.col("name").contains("Aero Park"), 'AK')
                                     .when(F.col("name").contains("Station") | F.col("name").contains("Air Station"), 'AS')
                                     .when(F.col("name").contains("Field")| F.col("name").contains("Fld"), 'FL')
                                     .otherwise('NaN'))

df_airports.groupBy('type').count().show()
df_airports.filter(F.col('type') == 'NaN').show()

+----+-----+
|type|count|
+----+-----+
|  AD|    1|
| NaN|  666|
|  AS|   19|
|  FL|   85|
|  AK|   12|
|  AP|  614|
+----+-----+

+---+--------------------+---------+-----------+----+----+---+-------------+----+
|faa|                name|      lat|        lon| alt|  tz|dst|       region|type|
+---+--------------------+---------+-----------+----+----+---+-------------+----+
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801|-6.0|  A|MAINLAND-EAST| NaN|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000|-5.0|  A|MAINLAND-EAST| NaN|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108|-8.0|  A|MAINLAND-WEST| NaN|
|1B9| Mansfield Municipal|42.000134|  -71.19677| 122|-5.0|  A|MAINLAND-EAST| NaN|
|3G3| Wadsworth Municipal| 41.00316|  -81.75644| 974|-5.0|  A|MAINLAND-EAST| NaN|
|49X|   Chemehuevi Valley| 34.52889| -114.43197| 638|-8.0|  A|MAINLAND-WEST| NaN|
|4U9|   Dell Flight Strip| 44.73575| -112.72002|6007|-7.0|  A|MAINLAND-WEST| NaN|
|6S2|            Florence| 43.98282| -124.11137| 

### Pergunta 6

In [18]:
MILITARY_LIST = ["Base", "Aaf", "AFs", "Ahp", "Afb", "LRRS", "Lrrs", "Arb",
                 "Naf", "NAS", "Nas", "Jrb", "Ns", "As", "Cgas", "Angb"]

REGEX_MILITARY = r'|'.join(map(lambda word: f'.*(^| ){word}( |$).*', MILITARY_LIST))

df_airports = df_airports.withColumn("military",
                                     F.when(F.col("name").rlike(REGEX_MILITARY), True)
                                     .otherwise(False))

df_airports.groupBy('military').count().show()
df_airports.printSchema()

+--------+-----+
|military|count|
+--------+-----+
|    true|  160|
|   false| 1237|
+--------+-----+

root
 |-- faa: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- lon: float (nullable = true)
 |-- alt: integer (nullable = true)
 |-- tz: float (nullable = true)
 |-- dst: string (nullable = true)
 |-- region: string (nullable = false)
 |-- type: string (nullable = false)
 |-- military: boolean (nullable = false)



### Pergunta 7

In [19]:
REGEX_ADMINISTRATION = {'I': None, 'N': None, 'R' : None, 'M' : None}
REGEX_ADMINISTRATION_LIST = [
    ["International", "Intl", "Intercontinental"],
    ["National", "Natl"],
    [ "Regional", "Reigonal", "Rgnl", "County", "Metro", "Metropolitan"],
    ["Municipal", "Muni", "City"]
]

for key, lista in zip(REGEX_ADMINISTRATION, REGEX_ADMINISTRATION_LIST):
    REGEX_ADMINISTRATION[key] = r'|'.join(map(lambda word: f'.*(^| ){word}( |$).*', lista))

df_airports = df_airports.withColumn("administration", 
                                     F.when(F.col('name').rlike(REGEX_ADMINISTRATION['I']), 'I')
                                    .when(F.col('name').rlike(REGEX_ADMINISTRATION['N']), 'N')
                                    .when(F.col('name').rlike(REGEX_ADMINISTRATION['R']), 'R')
                                    .when(F.col('name').rlike(REGEX_ADMINISTRATION['M']), 'M')
                                    .otherwise('NaN'))

df_airports.groupBy('administration').count().show()

+--------------+-----+
|administration|count|
+--------------+-----+
|             M|  179|
|             N|    5|
|             R|  281|
|           NaN|  768|
|             I|  164|
+--------------+-----+



## Dataset Planes

### Pergunta 1


In [20]:
df_planes = df_planes.withColumn("tailchar", 
                                 F.when(df_planes.tailnum.rlike('[A-Z]{2}$'),df_planes.tailnum.substr(-2,6))
                                 .when(df_planes.tailnum.rlike('[A-Z]$'), df_planes.tailnum.substr(-1,6)))
df_planes.select('tailnum','tailchar').filter(F.col('tailchar').isNull()).show()

df_planes.select('tailnum','tailchar').filter(F.length(F.col('tailnum')) == 5).show()

+-------+--------+
|tailnum|tailchar|
+-------+--------+
| N11206|    null|
| N12114|    null|
| N12216|    null|
| N12218|    null|
| N12221|    null|
| N12225|    null|
| N12238|    null|
| N13248|    null|
| N13716|    null|
| N13718|    null|
| N14214|    null|
| N14219|    null|
| N14228|    null|
| N14230|    null|
| N14231|    null|
| N14237|    null|
| N14242|    null|
| N14250|    null|
| N14704|    null|
| N14731|    null|
+-------+--------+
only showing top 20 rows

+-------+--------+
|tailnum|tailchar|
+-------+--------+
|  N1602|    null|
|  N1603|    null|
|  N1605|    null|
|  N1608|    null|
|  N1609|    null|
|  N3752|    null|
|  N3753|    null|
|  N3756|    null|
|  N3759|    null|
|  N3765|    null|
|  N3766|    null|
|  N3767|    null|
|  N3768|    null|
|  N6700|    null|
|  N6701|    null|
|  N6702|    null|
|  N6709|    null|
|  N704X|       X|
|  N744P|       P|
+-------+--------+



### Pergunta 2


In [21]:
df_planes.filter(df_planes.year == 0).show()

df_planes = df_planes.withColumn("year", 
                                 F.when(df_planes.year == 0, 1996)
                                 .otherwise(df_planes.year))

df_planes.filter(df_planes.year == 0).show()

+-------+----+--------------------+------------+---------+-------+-----+-----+----------+--------+
|tailnum|year|                type|manufacturer|    model|engines|seats|speed|    engine|tailchar|
+-------+----+--------------------+------------+---------+-------+-----+-----+----------+--------+
| N235SW|   0|Fixed wing multi ...|     EMBRAER|EMB-120ER|      2|   32| null|Turbo-prop|      SW|
+-------+----+--------------------+------------+---------+-------+-----+-----+----------+--------+

+-------+----+----+------------+-----+-------+-----+-----+------+--------+
|tailnum|year|type|manufacturer|model|engines|seats|speed|engine|tailchar|
+-------+----+----+------------+-----+-------+-----+-----+------+--------+
+-------+----+----+------------+-----+-------+-----+-----+------+--------+



### Pergunta 3

In [22]:
from pyspark.sql.window import Window

w = Window.partitionBy("manufacturer", "model").orderBy(F.col("manufacturer"), F.col("model"), F.col("year"))
w2 = Window.partitionBy("manufacturer").orderBy(F.col("manufacturer"), F.col("model"), F.col("year"))

df_planes.filter((F.col('model').like('A321-211') ) & (F.col('manufacturer').like('AIRBUS'))).orderBy('year').show()
df_planes.filter((F.col('model').like('737-824') ) & (F.col('manufacturer').like('BOEING'))).orderBy('year').show()


df_planes_year_not_null = df_planes.select('model','year','manufacturer').filter(df_planes.year.isNotNull())
df_planes_year_reference = df_planes_year_not_null.withColumn("row",F.row_number().over(w))
df_planes_year_reference.show()


df_planes_year_reference = (df_planes_year_reference
           .filter(df_planes_year_reference.row == 1)
           .drop("row")
          )
df_planes_year_reference = (df_planes_year_reference
           .withColumnRenamed("year","year_1")
           .withColumnRenamed("manufacturer","manufacturer_1")
          .withColumnRenamed("model","model_1"))

df_planes_1 = df_planes.join(df_planes_year_reference, 
                             (df_planes.manufacturer == df_planes_year_reference.manufacturer_1) &
                            (df_planes.model == df_planes_year_reference.model_1),
                             "left")

df_planes_year_reference = df_planes_year_not_null.withColumn("row",F.row_number().over(w2))
df_planes_year_reference = df_planes_year_reference.filter(df_planes_year_reference.row == 1).drop("row")
df_planes_year_reference = (df_planes_year_reference
           .withColumnRenamed("year","year_2")
           .withColumnRenamed("manufacturer","manufacturer_2")
            .withColumnRenamed("model", "model_2")
          )
df_planes_1 = df_planes_1.join(df_planes_year_reference, 
                               df_planes_1.manufacturer == df_planes_year_reference.manufacturer_2,
                               "left")

df_planes_1 = df_planes_1.withColumn("year",
                                     F.when(df_planes_1.year.isNull(), df_planes_1.year_1)
                                     .otherwise(df_planes_1.year))
df_planes_1 = df_planes_1.withColumn("year",
                                    F.when(df_planes_1.year.isNull(),df_planes_1.year_2)
                                     .otherwise(df_planes_1.year))

df_planes = df_planes_1.drop("year_1", "year_2",'model_1','model_2','manufacturer_1','manufacturer_2')


df_planes.filter(df_planes.year.isNull()).show()
df_planes.groupBy('manufacturer').count().show()
df_planes.filter(df_planes.tailnum.isin(['N194UW','N33292'])).show()
df_planes.count()


+-------+----+--------------------+------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|manufacturer|   model|engines|seats|speed|   engine|tailchar|
+-------+----+--------------------+------------+--------+-------+-----+-----+---------+--------+
| N194UW|null|Fixed wing multi ...|      AIRBUS|A321-211|      2|  199| null|Turbo-fan|      UW|
| N188US|2002|Fixed wing multi ...|      AIRBUS|A321-211|      2|  199| null|Turbo-fan|      US|
| N187US|2002|Fixed wing multi ...|      AIRBUS|A321-211|      2|  199| null|Turbo-fan|      US|
| N186US|2002|Fixed wing multi ...|      AIRBUS|A321-211|      2|  199| null|Turbo-fan|      US|
| N193UW|2008|Fixed wing multi ...|      AIRBUS|A321-211|      2|  199| null|Turbo-fan|      UW|
| N195UW|2008|Fixed wing multi ...|      AIRBUS|A321-211|      2|  199| null|Turbo-fan|      UW|
| N196UW|2009|Fixed wing multi ...|      AIRBUS|A321-211|      2|  199| null|Turbo-fan|      UW|
| N197UW|2009|Fixed wing multi

2628

### Pergunta 4

In [23]:
from datetime import datetime

hoje = datetime.now()
ano_atual = hoje.year

df_planes = df_planes.withColumn("age", ano_atual - df_planes.year)
df_planes.show()
df_planes.filter(df_planes.age.isNull()).show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+---+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|tailchar|age|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+---+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 24|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A32

### Pergunta 5

In [24]:
df_planes = df_planes.withColumn("type", 
                                 F.when(df_planes.type.startswith('Ro'), 'ROTORCRAFT')
                                 .when(df_planes.type.endswith('multi engine'), 'MULTI_ENG')
                                .when(df_planes.type.endswith('single engine'), 'SINGLE_ENG')
                                .otherwise(df_planes.type))
df_planes.select('type').groupBy('type').count().show()


+----------+-----+
|      type|count|
+----------+-----+
|SINGLE_ENG|   10|
| MULTI_ENG| 2615|
|ROTORCRAFT|    3|
+----------+-----+



### Pergunta 6


In [25]:
df_planes.select('manufacturer').groupBy("manufacturer").count().orderBy("manufacturer").show()

df_planes = df_planes.withColumn("manufacturer", 
                                 F.when(df_planes.manufacturer.startswith('AIR'),'AIRBUS')
                                 .when(F.col('manufacturer').startswith('MCD'), 'MCDONNELL DOUGLAS')
                                 .when(F.col('manufacturer').startswith('BARKER'),'BARKER JACK')
                                 .when(F.col('manufacturer').startswith('BOM'), 'BOMBARDIER')
                                 .when(F.col('manufacturer').startswith('CIRR'),'CIRRUS')
                                 .when(F.col('manufacturer').startswith('GU'),'GULFSTREAM')
                                 .when(F.col('manufacturer').startswith('RO'),'ROBINSON HELICOPTER')
                                 .otherwise(df_planes.manufacturer))

df_planes.select('manufacturer').groupBy("manufacturer").count().orderBy("manufacturer").show()
df_planes.count()

+--------------------+-----+
|        manufacturer|count|
+--------------------+-----+
|              AIRBUS|  397|
|    AIRBUS INDUSTRIE|  401|
|       BARKER JACK L|    1|
|                BELL|    1|
|              BOEING| 1460|
|      BOMBARDIER INC|  214|
|            CANADAIR|    8|
|              CESSNA|    4|
|  CIRRUS DESIGN CORP|    1|
|             EMBRAER|   37|
|GULFSTREAM AEROSPACE|    1|
|        KILDALL GARY|    1|
|     LAMBERT RICHARD|    1|
|          MARZ BARRY|    1|
|   MCDONNELL DOUGLAS|   94|
|MCDONNELL DOUGLAS...|    2|
|               PIPER|    2|
|ROBINSON HELICOPT...|    1|
|            SIKORSKY|    1|
+--------------------+-----+

+-------------------+-----+
|       manufacturer|count|
+-------------------+-----+
|             AIRBUS|  798|
|        BARKER JACK|    1|
|               BELL|    1|
|             BOEING| 1460|
|         BOMBARDIER|  214|
|           CANADAIR|    8|
|             CESSNA|    4|
|             CIRRUS|    1|
|            EMBRAER|   

2628

### Pergunta 7


In [26]:
df_planes.filter((F.col('model').contains('(')) | (F.col('model').contains(')'))).show()


df_planes = df_planes.withColumn("model", 
                                 F.when(F.col('model').contains('('), 
                                       F.regexp_replace('model',r'\(([^()]*)\)',''))
                                 .when(F.col('model').contains(')'), 
                                       F.regexp_replace('model',r'\(([^()]*)\)',''))
                                 .otherwise(F.col('model')))


df_planes.filter((F.col('model').contains('(')) | (F.col('model').contains(')'))).show()
df_planes.filter(F.col('model').contains('DC-9-82')).show()


+-------+----+---------+-----------------+--------------+-------+-----+-----+---------+--------+---+
|tailnum|year|     type|     manufacturer|         model|engines|seats|speed|   engine|tailchar|age|
+-------+----+---------+-----------------+--------------+-------+-----+-----+---------+--------+---+
| N426AA|1986|MULTI_ENG|MCDONNELL DOUGLAS|DC-9-82(MD-82)|      2|  172| null|Turbo-fan|      AA| 36|
| N434AA|1987|MULTI_ENG|MCDONNELL DOUGLAS|DC-9-83(MD-83)|      2|  172| null|Turbo-fan|      AA| 35|
| N436AA|1987|MULTI_ENG|MCDONNELL DOUGLAS|DC-9-83(MD-83)|      2|  172| null|Turbo-fan|      AA| 35|
| N437AA|1987|MULTI_ENG|MCDONNELL DOUGLAS|DC-9-83(MD-83)|      2|  172| null|Turbo-fan|      AA| 35|
| N438AA|1987|MULTI_ENG|MCDONNELL DOUGLAS|DC-9-83(MD-83)|      2|  172| null|Turbo-fan|      AA| 35|
| N439AA|1987|MULTI_ENG|MCDONNELL DOUGLAS|DC-9-83(MD-83)|      2|  172| null|Turbo-fan|      AA| 35|
| N456AA|1988|MULTI_ENG|MCDONNELL DOUGLAS|DC-9-82(MD-82)|      2|  172| null|Turbo-fan|    

### Pergunta 8


In [27]:
df_planes.filter(F.col('speed').isNull()).show()
df_planes = df_planes.withColumn("speed", 
                                 F.when(F.col('speed').isNull(), F.ceil(F.col('seats')/0.36))
                                .otherwise(F.col('speed')))


df_planes.filter(F.col('speed').isNull()).show()
df_planes.filter(F.col('tailnum') == 'N102UW').show()

+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+
|tailnum|year|     type|manufacturer|   model|engines|seats|speed|   engine|tailchar|age|
+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+
| N102UW|1998|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 24|
| N103US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N104UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N105UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N107US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N108UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N109UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N110UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N111US|1

### Pergunta 9

In [28]:
df_planes.select('engine').distinct().show()

df_planes = df_planes.withColumn('engine_type',
                                F.when(F.col('engine').endswith('cle'), 'CYCLE')
                                 .when(F.col('engine').endswith('aft'), 'SHAFT')
                                 .when(F.col('engine').endswith('fan'), 'FAN')
                                .when(F.col('engine').endswith('jet'),'JET')
                                .when(F.col('engine').endswith('rop'),'PROP'))
df_planes.groupBy('engine_type').count().show()
df_planes.filter(F.col('engine_type').isNull()).show()


+-------------+
|       engine|
+-------------+
|    Turbo-jet|
|      4 Cycle|
|    Turbo-fan|
|   Turbo-prop|
|Reciprocating|
|  Turbo-shaft|
+-------------+

+-----------+-----+
|engine_type|count|
+-----------+-----+
|       PROP|   37|
|       null|   10|
|      CYCLE|    1|
|        FAN| 2127|
|        JET|  450|
|      SHAFT|    3|
+-----------+-----+

+-------+----+----------+---------------+-------------+-------+-----+-----+-------------+--------+----+-----------+
|tailnum|year|      type|   manufacturer|        model|engines|seats|speed|       engine|tailchar| age|engine_type|
+-------+----+----------+---------------+-------------+-------+-----+-----+-------------+--------+----+-----------+
| N201AA|1959|SINGLE_ENG|         CESSNA|          150|      1|    2|   90|Reciprocating|      AA|  63|       null|
| N202AA|1980| MULTI_ENG|         CESSNA|         421C|      2|    8|   90|Reciprocating|      AA|  42|       null|
| N425AA|1968|SINGLE_ENG|          PIPER|    PA-28-180|   

## Flights Dataset

### Pergunta 1

In [29]:
print(df_flights.filter(F.col('minute').isNull()).count())
print(df_flights.filter(F.col('hour').isNull()).count())

df_flights = df_flights.withColumn("hour", 
                                   F.when((F.col('hour').isNull()) |
                                         F.col('hour').startswith('N') , 0)
                                  .otherwise(F.col('hour')))

df_flights = df_flights.withColumn("minute", 
                                   F.when((F.col('minute').isNull()) |
                                          F.col('minute').startswith('N'), 0) 
                                  .otherwise(F.col('minute')))

df_flights.filter(F.col('minute').isNull()).show()
df_flights.filter(F.col('hour').isNull()).show()
print(df_flights.filter(F.col('minute').isNull()).count())
print(df_flights.filter(F.col('hour').isNull()).count())
print(df_flights.filter(F.col('minute')==0).count())
print(df_flights.filter(F.col('hour')==0).count())

48
48
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+

0
0
226
137


### Pergunta 2


In [30]:
df_flights.filter(F.col('hour') == 24).show()

df_flights = df_flights.withColumn("hour", 
                                  F.when(F.col('hour') == 24, 0)
                                   .otherwise(F.col('hour')))

df_flights.filter(F.col('hour') == 24).show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|    6| 20|    2400|       10|     535|        0|     AA| N3GHAA|  2486|   SEA| ORD|     197|    1721|  24|     0|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
+----+-----+---+--------+---------+----

### Pergunta 3

In [31]:
df_flights = df_flights.withColumn('dep_string', 
                                   F.concat(
                                       F.col('year'),
                                       F.lit('-'),
                                       F.col('month'),
                                       F.lit('-'),
                                       F.col('day'),
                                       F.lit(' '),
                                       F.col('hour'),
                                       F.lit(':'),
                                       F.col('minute'),
                                       F.lit(':00')
                                   ))
df_flights = df_flights.withColumn('dep_datetime', F.to_timestamp(F.col('dep_string'), 'yyyy-M-d H:m:00'))
df_flights.select('dep_string','dep_datetime').show(30)
df_flights = df_flights.drop('dep_string')
df_flights.filter(df_flights.dep_datetime.isNull()).show()

+-------------------+-------------------+
|         dep_string|       dep_datetime|
+-------------------+-------------------+
|  2014-12-8 6:58:00|2014-12-08 06:58:00|
| 2014-1-22 10:40:00|2014-01-22 10:40:00|
|  2014-3-9 14:43:00|2014-03-09 14:43:00|
|   2014-4-9 17:5:00|2014-04-09 17:05:00|
|   2014-3-9 7:54:00|2014-03-09 07:54:00|
| 2014-1-15 10:37:00|2014-01-15 10:37:00|
|   2014-7-2 8:47:00|2014-07-02 08:47:00|
| 2014-5-12 16:55:00|2014-05-12 16:55:00|
| 2014-4-19 12:36:00|2014-04-19 12:36:00|
|2014-11-19 18:12:00|2014-11-19 18:12:00|
| 2014-11-8 16:53:00|2014-11-08 16:53:00|
|  2014-8-3 11:20:00|2014-08-03 11:20:00|
| 2014-10-30 8:11:00|2014-10-30 08:11:00|
|2014-11-12 23:46:00|2014-11-12 23:46:00|
|2014-10-31 13:14:00|2014-10-31 13:14:00|
|  2014-1-29 20:9:00|2014-01-29 20:09:00|
|2014-12-17 20:15:00|2014-12-17 20:15:00|
| 2014-8-11 10:17:00|2014-08-11 10:17:00|
| 2014-1-13 21:56:00|2014-01-13 21:56:00|
|  2014-6-5 17:33:00|2014-06-05 17:33:00|
|  2014-7-27 21:5:00|2014-07-27 21

### Pergunta 4


In [32]:
df_flights.select('dep_time').filter(df_flights.dep_time.rlike(REGEX_NOT_NUM)).distinct().show()
df_flights.filter(F.col('dep_time').isNull()).show()

df_flights = df_flights.withColumn('dep_time', 
                                   F.when(F.col('dep_time').endswith('A')
                                          , F.concat(F.col('hour'), 
                                                     F.when(F.length(F.col('minute'))== 1, 
                                                                     F.concat(F.lit('0'),F.col('minute')))
                                                     .otherwise(F.col('minute'))))
                                  .otherwise(F.col('dep_time')))
                                  
df_flights.select('dep_time').filter(df_flights.dep_time.rlike(REGEX_NOT_NUM)).distinct().show()
df_flights.filter(F.col('dep_time').isNull()).show()



+--------+
|dep_time|
+--------+
|      NA|
+--------+

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|dep_datetime|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+

+--------+
|dep_time|
+--------+
+--------+

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|dep_datetime|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--

### Pergunta 5


In [33]:
df_flights.filter(F.col('dep_delay').isNull()).show()
print(df_flights.filter(F.col('dep_delay').isNull()).count())
df_flights.filter(F.col('dep_delay').rlike(REGEX_NOT_NUM)).distinct().show()

df_flights = df_flights.withColumn('dep_delay', 
                                   F.when(F.col('dep_delay').isNull(), 0)
                                  .otherwise(F.col('dep_delay')))

df_flights.filter(F.col('dep_delay').isNull()).show()
print(df_flights.filter(F.col('dep_delay').isNull()).count())
df_flights.filter(F.col('dep_delay').rlike(REGEX_NOT_NUM)).show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|       dep_datetime|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|2014|    3|  4|     000|     null|      NA|     null|     UA|     NA|   156|   SEA| DEN|    null|    1024|   0|     0|2014-03-04 00:00:00|
|2014|    2| 12|     000|     null|      NA|     null|     AS| N527AS|     2|   SEA| DCA|    null|    2329|   0|     0|2014-02-12 00:00:00|
|2014|    7|  1|     000|     null|      NA|     null|     WN| N8323C|  2485|   SEA| MDW|    null|    1733|   0|     0|2014-07-01 00:00:00|
|2014|    4| 30|     000|     null|      NA|     null|     AS| N526AS|   566|   PDX| LAX|    null|     834|   0|     0|2014-04-30 00:00:00|
|2014|    1|  3|    

### Pergunta 6


In [34]:
df_flights.filter(F.col('arr_delay').isNull()).show()
print(df_flights.filter(F.col('arr_delay').isNull()).count())
df_flights.filter(F.col('arr_delay').rlike(REGEX_NOT_NUM)).distinct().show()

df_flights = df_flights.withColumn('arr_delay', 
                                  F.when(F.col('arr_delay').isNull(), 0)
                                  .otherwise(F.col('arr_delay')))

df_flights.filter(F.col('arr_delay').isNull()).show()
df_flights.filter(F.col('arr_delay').rlike(REGEX_NOT_NUM)).show()
print(df_flights.filter(F.col('dep_delay').isNull()).count())

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|       dep_datetime|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|2014|    4|  6|    1329|        4|    2159|     null|     DL| N130DL|  1929|   SEA| ATL|    null|    2182|  13|    29|2014-04-06 13:29:00|
|2014|    3|  4|     000|        0|      NA|     null|     UA|     NA|   156|   SEA| DEN|    null|    1024|   0|     0|2014-03-04 00:00:00|
|2014|    2| 12|     000|        0|      NA|     null|     AS| N527AS|     2|   SEA| DCA|    null|    2329|   0|     0|2014-02-12 00:00:00|
|2014|    7|  1|     000|        0|      NA|     null|     WN| N8323C|  2485|   SEA| MDW|    null|    1733|   0|     0|2014-07-01 00:00:00|
|2014|   12| 27|    

### Pergunta 7

In [35]:
df_flights = df_flights.drop('year','month','day','hour','minute')
df_flights.show()

+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+
|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|
+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+
|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|2014-12-08 06:58:00|
|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|2014-01-22 10:40:00|
|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|2014-03-09 14:43:00|
|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|2014-04-09 17:05:00|
|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|2014-03-09 07:54:00|
|    1037|        7|    1352|        2|     WN| N646SW|    48|   PDX| DEN|     121|     991|2014

### Pergunta 8

In [36]:
df_flights = df_flights.withColumn('air_time_projected', ((F.col('distance')*0.1) + 20).cast('int'))
df_flights.select('distance','air_time_projected').show()

+--------+------------------+
|distance|air_time_projected|
+--------+------------------+
|     954|               115|
|    2677|               287|
|     679|                87|
|     569|                76|
|     937|               113|
|     991|               119|
|     543|                74|
|     679|                87|
|    1050|               125|
|    1721|               192|
|     954|               115|
|    1107|               130|
|     867|               106|
|    1448|               164|
|     679|                87|
|     550|                75|
|     605|                80|
|    1733|               193|
|    2496|               269|
|     817|               101|
+--------+------------------+
only showing top 20 rows



### Pergunta 9


In [37]:
df_flights_avg_time = (df_flights.select('origin','dest','air_time')
                       .filter((~F.col('air_time').startswith('N')) & F.col('air_time').isNotNull())
                       )
df_flights_avg_time = df_flights_avg_time.withColumn('air_time', F.col('air_time').cast('int'))
df_flights_avg_time = df_flights_avg_time.groupBy("origin","dest").avg("air_time")

df_flights_avg_time = (df_flights_avg_time
                       .withColumnRenamed("origin","origin_1")
                       .withColumnRenamed("dest","dest_1"))


df_flights_avg_time.show()


df_flights = df_flights.join(df_flights_avg_time,
                            (F.col('origin') == F.col('origin_1')) &
                            (F.col('dest') == F.col('dest_1')),
                            "left")
df_flights = df_flights.withColumn('air_time_expected', F.col('avg(air_time)').cast('int'))
df_flights = df_flights.drop('dest_1','origin_1','avg(air_time)')
df_flights.show()

+--------+------+------------------+
|origin_1|dest_1|     avg(air_time)|
+--------+------+------------------+
|     SEA|   RNO|            74.375|
|     SEA|   DTW|219.81632653061226|
|     SEA|   CLE|             233.5|
|     SEA|   LAX| 126.8641425389755|
|     PDX|   SEA|34.927536231884055|
|     SEA|   BLI|              22.8|
|     PDX|   IAH|213.83928571428572|
|     PDX|   PHX|130.21153846153845|
|     SEA|   SLC| 88.98222222222222|
|     SEA|   SBA|118.30434782608695|
|     SEA|   BWI| 269.8965517241379|
|     PDX|   IAD| 267.8695652173913|
|     PDX|   SFO| 85.42809364548495|
|     SEA|   KOA|           347.075|
|     PDX|   MCI|174.46666666666667|
|     SEA|   SJC|103.13615023474179|
|     SEA|   ABQ|142.74418604651163|
|     SEA|   SAT|208.44444444444446|
|     PDX|   ONT| 111.6842105263158|
|     SEA|   LAS|118.46978021978022|
+--------+------+------------------+
only showing top 20 rows

+--------+---------+--------+---------+-------+-------+------+------+----+--------+---

### Pergunta 10


In [38]:
df_flights.filter(F.col('air_time').isNull()).show()
df_flights.filter(F.col('air_time').rlike(REGEX_NOT_NUM)).show()

df_flights = df_flights.withColumn('air_time', 
                                   F.when(F.col('air_time').isNull(),
                                          F.when(F.col('air_time_projected') > F.col('air_time_expected'),
                                                F.col('air_time_projected'))
                                          .otherwise(F.col('air_time_expected')))
                                   .otherwise(F.col('air_time')))

df_flights.filter(F.col('air_time').isNull()).show()

+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+-----------------+
|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|air_time_projected|air_time_expected|
+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+-----------------+
|    1329|        4|    2159|        0|     DL| N130DL|  1929|   SEA| ATL|    null|    2182|2014-04-06 13:29:00|               238|              245|
|     000|        0|      NA|        0|     UA|     NA|   156|   SEA| DEN|    null|    1024|2014-03-04 00:00:00|               122|              127|
|     000|        0|      NA|        0|     AS| N527AS|     2|   SEA| DCA|    null|    2329|2014-02-12 00:00:00|               252|              268|
|     000|        0|      NA|        0|     WN| N8323C|  2485|   SEA| MDW|    null|    1733|2014-07-

### Pergunta 11

In [39]:
df_flights.filter(F.col('arr_time').rlike(REGEX_NOT_NUM)).show()
df_flights.filter(F.col('arr_time').isNull()).show()
df_flights = df_flights.withColumn('dep_hour_teste',
                                  F.when(F.length(F.col('dep_time')) == 3,
                                        F.col('dep_time').substr(1,1).cast('int'))
                                  .when(F.length(F.col('dep_time')) == 4,
                                       F.col('dep_time').substr(1,2).cast('int')))

df_flights = df_flights.withColumn('dep_minute_teste',
                                  F.when(F.length(F.col('dep_time')) == 3,
                                        F.col('dep_time').substr(-2,3).cast('int'))
                                  .when(F.length(F.col('dep_time')) == 4,
                                       F.col('dep_time').substr(-2,4).cast('int')))

df_flights = df_flights.withColumn('dep_arr_total', F.col('dep_hour_teste')*60 + F.col('dep_minute_teste') + F.col('air_time'))
df_flights = df_flights.withColumn('horas', (F.col('dep_arr_total')/60).cast('int'))


df_flights = df_flights.withColumn('minutos', F.col('dep_arr_total') - (F.col('horas')*60))
df_flights = df_flights.withColumn('horas', 
                                   F.when(F.col('horas') == 24, 0)
                                  .otherwise(F.col('horas')))
df_flights = df_flights.withColumn('arr_time', 
                                  F.when(F.col('arr_time').startswith('N'),
                                         F.when(F.col('minutos') < 10, 
                                                F.concat(F.col('horas'),F.lit('0') ,F.col('minutos')))
                                         .otherwise(F.concat(F.col('horas'), F.col('minutos')))
                                        ) 
                                   .otherwise(F.col('arr_time')))
df_flights = df_flights.drop('dep_hour_teste','dep_minute_teste','dep_arr_total','horas','minutos')
df_flights.filter(F.col('arr_time').rlike(REGEX_NOT_NUM)).show()
df_flights.filter(F.col('tailnum') == 'N527AS').show()
df_flights.select('arr_time','air_time','dep_time').filter(F.col('arr_time').rlike(REGEX_NOT_NUM)).show(100)


+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+-----------------+
|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|air_time_projected|air_time_expected|
+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+-----------------+
|     000|        0|      NA|        0|     UA|     NA|   156|   SEA| DEN|     127|    1024|2014-03-04 00:00:00|               122|              127|
|     000|        0|      NA|        0|     AS| N527AS|     2|   SEA| DCA|     268|    2329|2014-02-12 00:00:00|               252|              268|
|     000|        0|      NA|        0|     WN| N8323C|  2485|   SEA| MDW|     214|    1733|2014-07-01 00:00:00|               193|              214|
|     000|        0|      NA|        0|     AS| N526AS|   566|   PDX| LAX|     114|     834|2014-04-

+--------+--------+--------+
|arr_time|air_time|dep_time|
+--------+--------+--------+
+--------+--------+--------+



### Pergunta 12

In [40]:
df_flights = df_flights.withColumn('haul_duration',
                                  F.when(F.col('air_time').between(20,180), 'SHORT-HAUL')
                                  .when(F.col('air_time').between(180,360), 'MEDIUM-HAUL')
                                  .when(F.col('air_time') > 360,'LONG-HAUL'))

df_flights.groupBy('haul_duration').count().show()
df_flights.select('air_time','haul_duration').filter(F.col('haul_duration').isNull()).show()

+-------------+-----+
|haul_duration|count|
+-------------+-----+
|    LONG-HAUL|   54|
|  MEDIUM-HAUL| 3254|
|   SHORT-HAUL| 6692|
+-------------+-----+

+--------+-------------+
|air_time|haul_duration|
+--------+-------------+
+--------+-------------+



### Pergunta 13

In [41]:
df_flights = df_flights.withColumn('dep_season',
                                  F.when(
                                      (F.col('dep_datetime') < F.concat(F.year('dep_datetime'),F.lit('-03-20 15:33:00'))) |
                                       (F.col('dep_datetime') > F.concat(F.year('dep_datetime'), F.lit('-12-21 21:48:00'))),
                                        'WINTER')
                                  .when(F.col('dep_datetime')
                                        .between(F.concat(F.year('dep_datetime'),F.lit('-03-20 15:33:01')),
                                                F.concat(F.year('dep_datetime'),F.lit('-06-21 10:14:00'))),
                                        'SPRING'
                                       )
                                   .when(F.col('dep_datetime')
                                        .between(F.concat(F.year('dep_datetime'),F.lit('-06-21 10:14:01')),
                                                F.concat(F.year('dep_datetime'),F.lit('-09-23 02:04:00'))),
                                        'SUMMER'
                                       )
                                    .when(F.col('dep_datetime')
                                        .between(F.concat(F.year('dep_datetime'),F.lit('-09-23 02:04:01')),
                                                F.concat(F.year('dep_datetime'),F.lit('-12-21 21:48:00'))),
                                        'FALL'
                                       )
                                  )

df_flights.select('dep_season').groupBy('dep_season').count().show()
df_flights.select('dep_datetime','dep_season').show()

+----------+-----+
|dep_season|count|
+----------+-----+
|    WINTER| 2149|
|    SPRING| 2560|
|      FALL| 2373|
|    SUMMER| 2918|
+----------+-----+

+-------------------+----------+
|       dep_datetime|dep_season|
+-------------------+----------+
|2014-12-08 06:58:00|      FALL|
|2014-01-22 10:40:00|    WINTER|
|2014-03-09 14:43:00|    WINTER|
|2014-04-09 17:05:00|    SPRING|
|2014-03-09 07:54:00|    WINTER|
|2014-01-15 10:37:00|    WINTER|
|2014-07-02 08:47:00|    SUMMER|
|2014-05-12 16:55:00|    SPRING|
|2014-04-19 12:36:00|    SPRING|
|2014-11-19 18:12:00|      FALL|
|2014-11-08 16:53:00|      FALL|
|2014-08-03 11:20:00|    SUMMER|
|2014-10-30 08:11:00|      FALL|
|2014-11-12 23:46:00|      FALL|
|2014-10-31 13:14:00|      FALL|
|2014-01-29 20:09:00|    WINTER|
|2014-12-17 20:15:00|      FALL|
|2014-08-11 10:17:00|    SUMMER|
|2014-01-13 21:56:00|    WINTER|
|2014-06-05 17:33:00|    SPRING|
+-------------------+----------+
only showing top 20 rows



### Pergunta 14

In [42]:
df_flights = df_flights.withColumn('dep_delay_category',
                                  F.when(F.col('dep_delay') < 0, 'ANTECIPATED')
                                   .when(F.col('dep_delay') == 0, 'INTIME')
                                   .when(F.col('dep_delay').between(0,59), 'MINOR')
                                   .when(F.col('dep_delay') >= 60 , 'MAJOR'))
df_flights.groupBy('dep_delay_category').count().show()

+------------------+-----+
|dep_delay_category|count|
+------------------+-----+
|       ANTECIPATED| 5894|
|             MAJOR|  395|
|             MINOR| 3065|
|            INTIME|  646|
+------------------+-----+



In [43]:
df_planes.write.mode('overwrite').parquet('output/planes_proc.parquet')
df_airports.write.mode('overwrite').parquet('output/airports_proc.parquet')
df_flights.write.mode('overwrite').parquet('output/flights_proc.parquet')